# TensorFlow Graph für OpenVino in IR Format konvertieren

## Einstellungen

In [ ]:
import tensorflow as tf
import os

# 
# workspace_dir = '/home/manuel/Bachelor_Arbeit/TensorFlow/workspace'

# pfad zu trainiertem dataset (enthällt trainings output)
# zB:
dataset_dir = os.path.join('/home/manuel/Bachelor_Arbeit/TensorFlow/workspace/OI_Animals_Augmented_9_3000')

# dataset = 'OI_Animals_LessAug_9_3000'

# name des trainings output ordners
model_output = 'faster_rcnn_inception_v2_coco_2018_01_28_outdropout'

# zu exporttierenden checkpoint festlegen:
checkpoint = 479766
is_faster_rcnn = True
is_inception = True

batch_size = 1
channel = 3

if is_faster_rcnn:
    img_size = 600
    json_config_file = '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/faster_rcnn_support_api_v1.14.json'
    pipeline = os.path.join(dataset_dir, model_output, 'faster_rcnn_inception_v2_coco.config')
else:
    img_size = 300
    json_config_file = '/opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_support_api_v1.14.json'
    if is_inception:
        pipeline = os.path.join(dataset_dir, model_output, 'ssd_inception_v2_coco.config')
    else:
        pipeline = os.path.join(dataset_dir, model_output, 'ssd_mobilenet_v2_coco.config')

    
pb_file = os.path.join(dataset_dir, model_output, 'exported/frozen_inference_graph.pb')

plugin_device = 'MYRIAD'

data_type = 'FP16'

input_shape_str = '['+ str(batch_size) +',' + str(img_size) + ',' + str(img_size) + ',' + str(channel) + ']'

output_dir = os.path.join(dataset_dir, model_output, 'open_vino')

mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'


## frozen graph exportieren

In [ ]:
checkpoint_str = os.path.join(dataset_dir, model_output, 'model.ckpt-' + str(checkpoint))
exported = os.path.join(dataset_dir, model_output, 'exported')

!python3 /home/manuel/Bachelor_Arbeit/TensorFlow/models/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path $pipeline \
    --trained_checkpoint_prefix $checkpoint_str \
    --output_directory $exported

##  in openvino IR Format konvertieren

In [ ]:
!python3 {mo_tf_path} \
--input_model {pb_file} \
--tensorflow_use_custom_operations_config {json_config_file} \
--input_shape {input_shape_str} \
--data_type {data_type} \
--input image_tensor \
--tensorflow_object_detection_api_pipeline_config {pipeline} \
--output_dir {output_dir}
